# Curation, selection and validation of 4CV frame labels

**Author(s):** Miguel Xochicale [@mxochicale](https://github.com/mxochicale)  
**Contributor(s):** Nhat Phug [@huynhatd13](https://github.com/huynhatd13) 

March2022; April2022; May 2022


## Introduction
This notebook presents prototypes to pre-process echocardiography datasets with the use of pytorch features. 

### Running notebook

1. Go to echocardiography repository path: `$HOME/repositories/echocardiography/`
2. Open echocardiography repo in pycharm and in the terminal type:
    ```
    git checkout master # or the branch
    git pull # to bring a local branch up-to-date with its remote version
    ```
3. Launch Notebook server
    Go to you repository path: `cd $HOME/repositories/echocardiography/scripts/dataloaders` and type in the pycharm terminal:
    ```
    conda activate rt-ai-echo-VE 
    jupyter notebook
    ```
    which will open your web-browser.
    
    
### References
* "Proposed Regulatory Framework for Modifications to Artificial Intelligence/Machine Learning (AI/ML)-Based Software as a Medical Device (SaMD) - Discussion Paper and Request for Feedback". https://www.fda.gov/media/122535/download 
* Gomez A. et al. 2021 https://github.com/vital-ultrasound/lung/blob/main/multiclass_pytorch/datasets/LUSVideoDataset.py 
* Save animation as gif (if required) or other formats https://holypython.com/how-to-save-matplotlib-animations-the-ultimate-guide/


## Jupyter Notebook
### Setting imports and datasets paths

In [ ]:
import os
import sys
import argparse
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
from IPython.display import HTML, display #to be used with HTML(animation.ArtistAnimation().to_jshtml())
from tqdm import tqdm
import shutil

import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

from typing import Dict, List, Tuple

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.utils.data as Data
from torchinfo import summary
from torchvision import transforms, utils, models

from source.dataloaders.EchocardiographicVideoDataset import EchoClassesDataset
from source.models.learning_misc import train_loop, test_loop
from source.helpers.various import concatenating_YAML_via_tags, plot_dataset_classes, split_train_validate_sets


HOME_PATH = os.path.expanduser(f'~')
USERNAME = os.path.split(HOME_PATH)[1]
CONFIG_FILES_PATH= 'repositories/echocardiography/scripts/config_files/users_paths_files'
YML_FILE =  'config_users_paths_files_username_' + USERNAME + '.yml'
FULL_PATH_FOR_YML_FILE = os.path.join(HOME_PATH, CONFIG_FILES_PATH, YML_FILE)
PATH_for_temporal_files = os.path.join(HOME_PATH, 'datasets/vital-us/echocardiography/temporal-files')

yaml.add_constructor('!join', concatenating_YAML_via_tags)  ## register the tag handler
with open(FULL_PATH_FOR_YML_FILE, 'r') as yml:
    config = yaml.load(yml, Loader=yaml.FullLoader)


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)    


### Splitting datasets

In [ ]:

## Setting ECHODATASET_PATH; 
#ECHODATASET_PATH = config['echodataset_path'] # Default
ECHODATASET_PATH = '/media/mx19/vitaluskcl/datasets/echocardiography/videos-echo-annotated-ALL'
#ECHODATASET_PATH = '/media/mx19/vitaluskcl/datasets/echocardiography/videos-echo-annotated-in-verification40-49'
#ECHODATASET_PATH = '/media/mx19/vitaluskcl/datasets/echocardiography/videos-echo-annotated-in-verification50-59'
#ECHODATASET_PATH = '/media/mx19/vitaluskcl/datasets/echocardiography/videos-echo-annotated-in-verification60-69'
#ECHODATASET_PATH = '/media/mx19/vitaluskcl/datasets/echocardiography/videos-echo-annotated-in-verification70-79'

split_train_validate_sets(  
                        ECHODATASET_PATH, #config['echodataset_path']
                        config['data_list_output_path'], 
                        config['ntraining'],
                        config['randomise_file_list']
                        )


# Verification of participants
### Setting up variables for participants

In [ ]:
SUBJECT_ID = '053'
NUMBER_OF_FRAMES_PER_SEGMENT_IN_A_CLIP = 500
PRETRANSFORM_IM_SIZE = [200, 200] #[650, 690] original pixel size for VenueGO

### Other relevant variables

In [ ]:
interval_between_frames_in_milliseconds=33.3 ## 1/30=0.033333
frame_per_seconds_for_animated_frames=30

###  Setting up txt list for participants

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #or "cuda:0"

videolist = config['participant_videos_list_full']
annotationlist = config['participant_path_json_list_full']

video_filenames = [line.strip() for line in open(videolist)]
annotation_filenames = [line.strip() for line in open(annotationlist)]

data_list_output_path = config['data_list_output_path']
#print(data_list_output_path)

all_videos_files_to_be_verified = 'video_list_tbv.txt'
all_annotation_files_to_be_verified = 'annotation_list_tbv.txt'
videolist_tbv_txt = '{}{}'.format(data_list_output_path, all_videos_files_to_be_verified)
annotationlist_tbv_txt = '{}{}'.format(data_list_output_path, all_annotation_files_to_be_verified)

JSON_FILE_PATTERN = '*'+SUBJECT_ID+'*4[cC][vV].[jJ][sS][oO][nN]'
MP4_FILE_PATTERN = '*'+SUBJECT_ID+'*echo*.[mM][pP][4]'

result = list(sorted(Path(ECHODATASET_PATH).rglob(MP4_FILE_PATTERN)))
with open(videolist_tbv_txt, 'w') as file_list_txt:
    for mp4_filename_i in result:
        file_n_nopath = str(mp4_filename_i).replace(ECHODATASET_PATH, '')
        file_list_txt.write(file_n_nopath + '\n')
        

result = list(sorted(Path(ECHODATASET_PATH).rglob(JSON_FILE_PATTERN)))
with open(annotationlist_tbv_txt, 'w') as file_list_txt:
    for mp4_filename_i in result:
        file_n_nopath = str(mp4_filename_i).replace(ECHODATASET_PATH, '')
        file_list_txt.write(file_n_nopath + '\n')        
        

### Setting variables and loading datasets using pytorch dataloader

In [ ]:
# Defining transforms that apply to the entire dataset.
# These transforms are not augmentation.
if config['use_pretransform_image_size']:
    pretransform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size=PRETRANSFORM_IM_SIZE),
        transforms.ToTensor(),
    ])
else:
    pretransform = None

# These transforms have random parameters changing at each epoch.
if config['use_train_augmentation']:
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=5),  # in degrees
        transforms.RandomEqualize(p=0.5),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(), 
    ])
else:
    transform = None
    

echo_dataset = EchoClassesDataset(
    echodataset_path=ECHODATASET_PATH,
    temporal_data_path=config['temporal_data_path'],
    participant_videos_list=config['participant_videos_list_tbv'],
    participant_path_json_list=config['participant_path_json_list_tbv'],
    crop_bounds_for_us_image=config['crop_bounds_for_us_image'],
    pretransform_im_size=PRETRANSFORM_IM_SIZE,
    pretransform=pretransform,
    number_of_frames_per_segment_in_a_clip=NUMBER_OF_FRAMES_PER_SEGMENT_IN_A_CLIP,
    sliding_window_length_in_percentage_of_frames_per_segment=config['sliding_window_length_in_percentage_of_frames_per_segment'],
    device=DEVICE,
    max_background_duration_in_secs=config['max_background_duration_in_secs'],
    transform=None,
    use_tmp_storage=config['use_tmp_storage'],
    )


### Plotting Class Distribution


In [ ]:
label_id = ('BKGR', '4CV')
number_of_frames_per_segment_in_a_clip = config['number_of_frames_per_segment_in_a_clip'] 

def get_class_distribution(dataset_obj):
    count_class_dict = {
   'BKGR': 0 ,
   "4CV": 0
    }
    
    for clip_index_i in range(len(dataset_obj)):
        data_idx = dataset_obj[clip_index_i]
        label_id_idx = data_idx[1]
        label = label_id[label_id_idx]
        count_class_dict[label]+= 1
        #count_class_dict[label]+= 1* number_of_frames_per_segment_in_a_clip

    return count_class_dict
        
        
def plot_from_dict(dict_obj, plot_title, **kwargs):
    return sns.barplot(data = pd.DataFrame.from_dict([dict_obj]).melt(), 
                       x = "variable", y="value", hue="variable", **kwargs).set_title(plot_title)

print(get_class_distribution(echo_dataset))

plot_title_train_label= f'TRAIN dataset of {len(echo_dataset)} clips with {NUMBER_OF_FRAMES_PER_SEGMENT_IN_A_CLIP} n_frames_per_clip'
#fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(18,7))
plot_from_dict(get_class_distribution(echo_dataset), plot_title=plot_title_train_label)
plt.show()

### Animating frames of one clip of the dataloader

In [ ]:
#average_HR =
#fps = 30
# 60 # beats per minute 
#Beats-per-minute: 60 BPM
#Beats-per-second: 1 Hz
#Cycle-per-second: 1 (Cycle/s)

number_of_clips = len(echo_dataset)


print(f'---------------------------------------')
### Setting clips
clips=[]
for clip_index in range( int(number_of_clips)  ):
    data_idx = echo_dataset[clip_index]
    data_clip_idx = data_idx[0]
    label_clip_idx = data_idx[1]
    clip_frame_clip_idx = data_idx[2]
    n_available_frames_clip_idx = data_idx[3]
    print(f' CLIP:{clip_index:02d} of {label_id[label_clip_idx]} label for {data_clip_idx.size()} TOTAL_FRAMES: {n_available_frames_clip_idx} from clip_frame_clip_idx {clip_frame_clip_idx}')    
    clips.append([data_clip_idx, label_clip_idx, clip_index, clip_frame_clip_idx, n_available_frames_clip_idx ]) 


# ### Setting and displaying pair of clips    
def pair_clips_labels(clips):
    pair_clips_labels_=[]    
    number_of_clips=len(clips)
    for clip_index_i_A in range( int(number_of_clips/2)  ):
        clip_index_i_B=int(number_of_clips/2) + clip_index_i_A 
        print(f' pair_clips_labels[{clip_index_i_A}]-- BKRG:{clip_index_i_A}, 4CV:{clip_index_i_B}')
        data_clip_i_A=clips[clip_index_i_A][0]
        label_i_A=clips[clip_index_i_A][1]
        clip_i_A=clips[clip_index_i_A][2]
        number_of_frames_A=clips[clip_index_i_A][4]
        data_clip_i_B=clips[clip_index_i_B][0]
        label_i_B=clips[clip_index_i_B][1]
        clip_i_B=clips[clip_index_i_B][2]
        number_of_frames_B=clips[clip_index_i_B][4]
        pair_clips_labels_.append([data_clip_i_A, label_i_A, clip_i_A, number_of_frames_A, data_clip_i_B, label_i_B, clip_i_B, number_of_frames_B])
    
    return(pair_clips_labels_)


print(f'---------------------------------------')
pair_clips_labels = pair_clips_labels(clips)

### Defining animate clips
def animate_clips(pair_clips_labels):
    #print(f' CLIP: for {label_id[pair_clips_labels[1]]} ')
    fig = plt.figure()    
    pair_of_clip_index_i_frames=[]   
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,5))
    data_clip_tensor_A=pair_clips_labels[0]
    label_clip_A=pair_clips_labels[1]
    clip_i_A=pair_clips_labels[2]
    number_of_frames_A=pair_clips_labels[3]
    data_clip_tensor_B=pair_clips_labels[4]
    label_clip_B=pair_clips_labels[5]
    clip_i_B=pair_clips_labels[6]
    number_of_frames_B=pair_clips_labels[7]
    
    
    ax1.title.set_text(f' CLIP: {clip_i_A:02d}--{label_id[label_clip_A]} with {number_of_frames_A} of {NUMBER_OF_FRAMES_PER_SEGMENT_IN_A_CLIP} frames [for Subject {SUBJECT_ID}]')
    ax2.title.set_text(f' CLIP: {clip_i_B:02d}--{label_id[label_clip_B]} with {number_of_frames_B} of {NUMBER_OF_FRAMES_PER_SEGMENT_IN_A_CLIP} frames [for Subject {SUBJECT_ID}]')
    for frames_idx in range(data_clip_tensor_A[:,:,...].size()[1]):
        imA = ax1.imshow(data_clip_tensor_A[:,frames_idx,...].cpu().detach().numpy().transpose(1,2,0) , cmap=plt.get_cmap('gray') )  
        imB = ax2.imshow(data_clip_tensor_B[:,frames_idx,...].cpu().detach().numpy().transpose(1,2,0) , cmap=plt.get_cmap('gray') )  
        pair_of_clip_index_i_frames.append( [imA, imB] )
    fig.tight_layout()    
    #return fig, pair_of_clip_index_i_frames

    anim = animation.ArtistAnimation(fig, pair_of_clip_index_i_frames, interval=interval_between_frames_in_milliseconds, blit=True, repeat_delay=1000)
    return anim

# plt.show()    
###BLURS
#
#fig, ax = plt.subplots()
#clips_axes.append( clip_index_i_frames )
# #plt.axes()
# #plt.off()

In [ ]:
##SAVE ANIMATIONS
for idx in range(0,len(pair_clips_labels)):
    PAIR_OF_CLIPS = pair_clips_labels[idx]
    print( f' pair_clips_labels {str(PAIR_OF_CLIPS[2])} {str(PAIR_OF_CLIPS[6])}')

    animated_frames=animate_clips(PAIR_OF_CLIPS)
    HTML(animated_frames.to_jshtml())   

    GIF_FILE='subject'+SUBJECT_ID+'_clips_'+str(PAIR_OF_CLIPS[2])+'-'+str(PAIR_OF_CLIPS[6])+'.gif'    
    GIF_FILENAME_FULLPATH = os.path.join(PATH_for_temporal_files, GIF_FILE)
    animated_frames.save(GIF_FILENAME_FULLPATH, writer=animation.PillowWriter(fps=frame_per_seconds_for_animated_frames))
    

### Displayting animated frames in the dataloader
**NOTE:** you might need to uncommed the lines where there is no clips

## `pair_clips_labels[0]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
PAIR_OF_CLIPS = pair_clips_labels[0]

animated_frames=animate_clips(PAIR_OF_CLIPS)
HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[1]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[1]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[2]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[2]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[3]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[3]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[4]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[4]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[5]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[5]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[6]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[6]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[7]`
**NOTE:** _you might need to comment this line as won't be available for_

In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[7]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


## `pair_clips_labels[8]`
**NOTE:** _you might need to comment this line as won't be available for_


In [ ]:
# PAIR_OF_CLIPS = pair_clips_labels[8]

# animated_frames=animate_clips(PAIR_OF_CLIPS)
# HTML(animated_frames.to_jshtml())   


# List of verified files 

**Instructions:** 
Tick the box once it is verified. If there are comments, please add them at the end of each line.
See two examples of patient 40 and 41. 


* =====================================================================
* =====================================================================
* =====================================================================
* [x] 040 - T1-03clips; 
* [x] 040 - T2-02clips; 
* [x] 040 - T3-00clips; 
* [x] Comments: 
* ---
* [x] 041 - T1-02clips; 
* [x] 041 - T2-02clips; 
* [x] 041 - T3-04clips; 
* [x] Comments:
* ---
* [x] 042 - T1-01clips; 
* [x] 042 - T2-01clips; 
* [x] 042 - T3-03clips; 
* [x] Comments: 
* ---
* [x] 043 - T1-01clips; 
* [x] 043 - T2-02clips; 
* [x] 043 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 044 - T1-00clips; 
* [x] 044 - T2-03clips; 
* [x] 044 - T3-01clips; 
* [x] Comments: 
* ---
* [x] 045 - T1-02clips; 
* [x] 045 - T2-01clips; 
* [x] 045 - T3-03clips; 
* [x] Comments: 4CVs don't look good: subject045_clips_2-8.gif; subject045_clips_3-9.gif; subject045_clips_4-10.gif
* [x] Comments: Renamed filenames
* ---
* [x] 046 - T1-02clips; 
* [x] 046 - T2-02clips; 
* [x] 046 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 047 - T1-03clips; 
* [x] 047 - T2-02clips; 
* [x] 047 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 048 - T1-02clips; 
* [x] 048 - T2-01clips; 
* [x] 048 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 049 - T1-??clips; 
* [x] 049 - T2-??clips; 
* [x] 049 - T3-??clips; 
* [x] Comments: NO_VIDEOS
* [x] Comments: TODO: No annotated!
* =====================================================================
* =====================================================================
* =====================================================================
* [x] 050 - T1-02clips; 
* [x] 050 - T2-02clips; 
* [x] 050 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 051 - T1-02clips; 
* [x] 051 - T2-02clips; 
* [x] 051 - T3-01clips; 
* [x] Comments: 
* ---
* [x] 052 - T1-02clips; 
* [x] 052 - T2-01clips; 
* [x] 052 - T3-00clips; 
* [x] Comments: MX: 4CV doesn't look good to me! subject052_clips_0-3.gif; subject052_clips_2-5.gif
* [x] Comments: TODO: Nhat will re-annotate labels
* ---
* [x] 053 - T1-04clips; 
* [x] 053 - T2-02clips; 
* [x] 053 - T3-03clips; 
* [x] Comments: 4CV don't look good: subject053_clips_6-15.gif; subject053_clips_7-16.gif; subject053_clips_8-17.gif
* [x] Comments: json files were duplicated and therefore delete one and delete one mp4!
* ---
* [x] 054 - T1-02clips; 
* [x] 054 - T2-00clips; 
* [x] 054 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 055 - T1-??clips; 
* [x] 055 - T2-??clips; 
* [x] 055 - T3-??clips; 
* [x] Comments: NO_VIDEOS
* [x] Comments: Confirmed NO VIDEOS
* ---
* [x] 056 - T1-00clips; 
* [x] 056 - T2-01clips; 
* [x] 056 - T3-01clips; 
* [x] Comments: 
* ---
* [x] 057 - T1-00clips; 
* [x] 057 - T2-01clips; 
* [x] 057 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 058 - T1-01clips; 
* [x] 058 - T2-02clips; 
* [x] 058 - T3-03clips; 
* [x] Comments: 
* ---
* [x] 059 - T1-??clips; 
* [x] 059 - T2-??clips; 
* [x] 059 - T3-??clips; 
* [x] Comments: NO_VIDEOS
* [x] Comments: Confirmed NO_VIDEOS
* =====================================================================
* =====================================================================
* =====================================================================
* [x] 060 - T1-01clips; 
* [x] 060 - T2-00clips; 
* [x] 060 - T3-00clips;
* [x] Comments: 4CVs don't look good: subject060_clips_0-1.gif
* [x] Comments: 4CVs is OKAY as there are many factors that are considered in the quality
* ---
* [x] 061 - T1-00clips; 
* [x] 061 - T2-00clips; 
* [x] 061 - T3-01clips;
* [x] Comments: 
* ---
* [x] 062 - T1-??clips; 
* [x] 062 - T2-??clips; 
* [x] 062 - T3-??clips;
* [x] Comments: NO_VIDEOS
* [x] Comments: Confirmed NO_VIDEOS
* ---
* [x] 063 - T1-02clips; 
* [x] 063 - T2-02clips; 
* [x] 063 - T3-01clips;
* [x] Comments:  4CVs don't look good: subject063_clips_2-7.gif
* [x] Comments: It's fine but has lung artifacts
* ---
* [x] 064 - T1-02clips; 
* [x] 064 - T2-03clips; 
* [x] 064 - T3-00clips;
* [x] Comments: 4CVs don't look good to me: subject064_clips_0-5.gif; subject064_clips_1-6.gif; subject064_clips_2-7.gif; subject064_clips_3-8.gif; subject064_clips_4-9.gif
* [x] Comments: 4CVs contain lung artifacts which is fine
* ---
* [x] 065 - T1-03clips; 
* [x] 065 - T2-04clips; 
* [x] 065 - T3-05clips;
* [x] Comments: 
* ---
* [x] 066 - T1-01clips; 
* [x] 066 - T2-02clips; 
* [x] 066 - T3-00clips;
* [x] Comments: 
* ---
* [x] 067 - T1-??clips; 
* [x] 067 - T2-??clips; 
* [x] 067 - T3-??clips;
* [x] Comments: NO_VIDEOS
* [x] Comments: Confirmed NO_VIDEOS
* ---
* [x] 068 - T1-02clips; 
* [x] 068 - T2-00clips; 
* [x] 068 - T3-02clips;
* [x] Comments: 
* ---
* [x] 069 - T1-02clips; 
* [x] 069 - T2-02clips; 
* [x] 069 - T3-02clips;
* [x] Comments: 
* =====================================================================
* =====================================================================
* =====================================================================
* [x] 070 - T1-04clips; 
* [x] 070 - T2-04clips; 
* [x] 070 - T3-01clips; 
* [x] Comments: 4CVs don't look good to me: subject070_clips_4-13.gif; subject070_clips_5-14.gif; subject070_clips_6-15.gif; subject070_clips_7-16.gif
* [x] Comments: TODO: Annotarions are wrong! It needs to be re-annotated!
* ---
* [x] 071 - T1-01clips; 
* [x] 071 - T2-02clips; 
* [x] 071 - T3-01clips; 
* [x] Comments: 
* ---
* [x] 072 - T1-01clips; 
* [x] 072 - T2-03clips; 
* [x] 072 - T3-02clips
* [x] Comments: 4CVs don't look good to me: subject072_clips_4-10.gif; subject072_clips_5-11.gif
* [x] Comments: No 4CV for clips in T3
* ---
* [x] 073 - T1-02clips; 
* [x] 073 - T2-01clips; 
* [x] 073 - T3-00clips; 
* [x] Comments: 
* ---
* [x] 074 - T1-02clips; 
* [x] 074 - T2-02clips; 
* [x] 074 - T3-00clips; 
* [x] Comments: 
* ---
* [x] 075 - T1-01clips; 
* [x] 075 - T2-02clips; 
* [x] 075 - T3-02clips; 
* [x] Comments: Duplicated 4CVs clips: subject075_clips_1-6.gif; subject075_clips_2-7.gif
* [x] Comments: It is not duplicated! So, clips are good!
* ---
* [x] 076 - T1-01clips; 
* [x] 076 - T2-01clips; 
* [x] 076 - T3-02clips; 
* [x] Comments: 
* ---
* [x] 077 - T1-00clips; 
* [x] 077 - T2-00clips; 
* [x] 077 - T3-00clips; 
* [x] Comments: VIDEOS BUT NO 4CV CLIPS
* ---
* [x] 078 - T1-01clips; 
* [x] 078 - T2-01clips; 
* [x] 078 - T3-01clips; 
* [x] Comments: 4CVs don't look good to me: subject078_clips_0-3.gif
* [x] Comments: 4CVs looks good but perhaps not at the best quality!
* ---
* [x] 079 - T1-01clips; 
* [x] 079 - T2-02clips; 
* [x] 079 - T3-02clips; 
* [x] Comments: 
* ---

In [ ]:
BATCH_SIZE_OF_CLIPS = 4

print(f' echo_dataset.__len__() = {echo_dataset.__len__()}')
echo_dataloader = torch.utils.data.DataLoader(
    echo_dataset, 
    batch_size=BATCH_SIZE_OF_CLIPS, 
    shuffle=True, 
    num_workers=0)


for clip_batch_idx, sample_batched in enumerate(echo_dataloader):
    print(f'====================================================')
    sample_batched_images=sample_batched[0]
    sample_batched_labels=sample_batched[1]
    print(f'BATCH_OF_CLIPS_INDEX: {clip_batch_idx} ')
    print(f'SAMPLE_IDX_LABELS: {  sample_batched_labels  }')
    print(f'SAMPLE_BATCH: {sample_batched_images.size()}')
    
    sample_batched=sample_batched_images #.squeeze()
    print(f'SAMPLE_BATCH.squeeze: {sample_batched.size()}')
    
    for BATCH_SIZE_IDX, label in enumerate(sample_batched_labels):
        print(f'   CLIP_BATCH_SIZE_IDX {BATCH_SIZE_IDX} label: {label}')
        sample_batched_idx_image = sample_batched[BATCH_SIZE_IDX,...]
        print(f'   Sample_batched_idx_image.size()  {sample_batched_idx_image.size() }'  )
        
        grid = utils.make_grid(sample_batched_idx_image)
        print(f'   Grid size {grid.size()}' )
         #plt.figure(figsize =(20,20) )
         #plt.imshow( grid.cpu().detach().numpy().transpose(1,2,0) )
         #plt.title(f'BATCH_SIZE_IDX {BATCH_SIZE_IDX}; Label: {label_id[label]}')
         #plt.axis('off')
         #plt.ioff()
         #plt.show()



## TO BE VERIFIED




* [ ] 080 - T1-00clips; 
* [ ] 080 - T2-00clips; 
* [ ] 080 - T3-00clips
* =======================
* [ ] 081 - T1-00clips; 
* [ ] 081 - T2-00clips; 
* [ ] 081 - T3-00clips


In [ ]:
## TODO: For better quality control of subject verifcation
# subjectsCSV_PATH = 'datasets/vital-us/echocardiography/subjectsCSV/anonimised'
# CSV_FILE = 'validation_anonymised_april2022.csv'
# FULL_PATH_FOR_CSV_FILE = os.path.join(HOME_PATH, subjectsCSV_PATH, CSV_FILE)
# datatablea_validation_anonymised = pd.read_csv(FULL_PATH_FOR_CSV_FILE)

# ## Filtering columns
# basic_demographics=datatablea_validation_anonymised.filter(items=[ 'SUBJID', 'LABELLED', 'CLIPS_DAY01', 'CLIPS_DAY02', 'CLIPS_DAY02'])
# basic_demographics

# print(f'=================== LABELLED =======================')
# basic_demographics['LABELLED'].value_counts().plot.pie(autopct='%.1f %%', ylabel='TOTAL', legend=True)
# plt.axis('equal')
# plt.show()


###SAVE ANIMATIONS
# ### Save animation as gif (if required) or other formats https://holypython.com/how-to-save-matplotlib-animations-the-ultimate-guide/
# f = r"/home/mx19/repositories/echocardiography/scripts/learning-pipeline/animation.gif" 
# writergif = animation.PillowWriter(fps=30) 
# writergif2='imagemagick'
# anim.save(f, dpi=80, writer=writergif)



## [**!WARNING!**] Cleanup temporal data directory 
Remove directory if a temporary was used.



```
       Make sure you know which path you will remove as you do not like to remove important files.
       shutil.rmtree
```

In [ ]:
# temporal_files_path = config['temporal_data_path']

# shutil.rmtree(temporal_files_path)
# print(f' {temporal_files_path} is empty')